### Libaries

In [1]:
import pprint
import sys
import json
import spotipy
import spotipy.util as util
import pprint
import pandas as pd
from pandas.io.json import json_normalize
import math
import time
import numpy as np
sleep_min = 1
sleep_max = 5

### Connect to Spotify API

Here we need different parameters to connect to the API, 
we need an OAuth2 authorization but also a Token to get current user data.

The scope is a Spotify parameter to have access to different parts of the API

The Client_Id and Client_Secret are provided by Spotify when creating a 'New App' 
at https://developer.spotify.com/dashboard/applications

The first time it connects OAUTH2 answer a token to a locahost:8080, you have to copy the website link and run the code. This option is not possible under Jupyter notebook so run the file file://../../connect_spotify_api.py and follow the steps.

After the response, it creates a file .cache_['username'] wich will allow access for some time.

In [2]:
username = 'nerken' # 'incautiouswifi' #some users don't have a name, just an ID
scope = 'user-follow-read,user-follow-modify,user-read-recently-played,user-top-read,user-library-read,user-library-modify,user-read-playback-state,user-read-currently-playing,user-modify-playback-state,playlist-read-collaborative,playlist-modify-private,playlist-modify-public,playlist-read-private,app-remote-control,user-read-email,user-read-private'

client_id = '22e782a724264812b124038bbf783396'
client_secret = '6219554f45764f2cbfe13306ec476f10'
oauth2 = util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
token = util.prompt_for_user_token(username, scope=scope, client_id='22e782a724264812b124038bbf783396', client_secret='6219554f45764f2cbfe13306ec476f10', redirect_uri='https://localhost:8080')

In [3]:
# Let's test if we have a OAuth2 and the Token
print(f'The Oauth2 object is: {oauth2} \n')
print(f'The current user token is: {token}')

The Oauth2 object is: <spotipy.oauth2.SpotifyClientCredentials object at 0x000001EA5B8D9148> 

The current user token is: BQB6naERgoFKuSoHy2-mrq-MDlmhaZMRT42XTLa5ZtEbBy7MYZy1AatxarUgUfFoxrbVKTERgeaMivD7NgSrcs7u7Qvga7Q31TP4kwcGu9UiT7eubaj35xFzD4CRZ1Nv6uhvSID4UWKqSzXYDJbkYNiXBpZ4gwku6SRG6Nci4LYHZerTRbJvE7bfoJ1teMGxqsFGbhJbNF0Z_5aZURRKre6octmSq28Uzse2tjlNdKIww5VMRRm0tazdMH0XRpwJaurF4DGleEst


### Getting the User Data

Here we want to get all the playlist for the current user, and split it between Album, Artist, Playlist, Songs. Since all the data is well structurated on spotify API the process will be from playlist->Album->Artist->Song.

Firstly I will get all the raw data and save it to csv to be able to wrangling it, cleaning it and upload it to a database.

In [4]:
# open a Spotify connection
sp = spotipy.Spotify(auth=token)

In [5]:
# let's get user information even we probably not gonna use it
user_data = sp.user(user=username)
json_normalize(user_data,max_level=1)

,display_name,href,id,images,type,uri,external_urls.spotify,followers.href,followers.total
0,Josep Foradada,https://api.spotify.com/v1/users/nerken,nerken,"[{'height': None, 'url': 'https://scontent.xx....",user,spotify:user:nerken,https://open.spotify.com/user/nerken,None,26


In [6]:
# Get total user playlists
total_user_playlists = json_normalize(sp.user_playlists(user=username,limit=1,offset=0),max_level=0).total[0]

In [7]:
total_user_playlists

74

In [8]:
#generate a function that iterates to get all playlists with a time delay for each request

def get_user_playlists(username,total_user_playlists):
        '''
        Get all playlists from a current user
        '''
        limit = 50 # number of results per request 
        loops = math.ceil(total_user_playlists/limit)
        users_playlists = pd.DataFrame()
        for x in range(loops):
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            results = sp.current_user_playlists(limit=50,offset=0+(x*50))
            for x in range(len(results['items'])):
                a = (json_normalize(results['items'][x]))
                users_playlists = users_playlists.append(a, ignore_index=True)
        return users_playlists
        

In [9]:
users_playlists = get_user_playlists(username,total_user_playlists)

In [10]:
users_playlists.head(5)

,collaborative,href,id,images,name,primary_color,public,snapshot_id,type,uri,external_urls.spotify,owner.display_name,owner.external_urls.spotify,owner.href,owner.id,owner.type,owner.uri,tracks.href,tracks.total
0,True,https://api.spotify.com/v1/playlists/0ogOkIjBi...,0ogOkIjBiisl0aqtx53DHY,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",Atún de la Frontera🐟,None,False,MTQ5LGRiYTMzOGYyN2Q2ZDg1NDlkODRiMmU1YTBjYmMxYz...,playlist,spotify:playlist:0ogOkIjBiisl0aqtx53DHY,https://open.spotify.com/playlist/0ogOkIjBiisl...,Íngrid Munné Collado,https://open.spotify.com/user/wobniar,https://api.spotify.com/v1/users/wobniar,wobniar,user,spotify:user:wobniar,https://api.spotify.com/v1/playlists/0ogOkIjBi...,121
1,False,https://api.spotify.com/v1/playlists/3tW0YVAZt...,3tW0YVAZtrpV0oR7lxQxxy,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",RUMBA CATALANA,None,False,MjUwLDk3YTk3Zjk0ZDg4ZTY1NjhlNGY0YzQyODNlNGM2ZD...,playlist,spotify:playlist:3tW0YVAZtrpV0oR7lxQxxy,https://open.spotify.com/playlist/3tW0YVAZtrpV...,joansayeras,https://open.spotify.com/user/joansayeras,https://api.spotify.com/v1/users/joansayeras,joansayeras,user,spotify:user:joansayeras,https://api.spotify.com/v1/playlists/3tW0YVAZt...,163
2,False,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DWUAZoWydCivZ,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Positive Vibes,None,False,MTU2MzQzMDI2MCwwMDAwMDAzMjAwMDAwMTZjMDNiNDhlZD...,playlist,spotify:playlist:37i9dQZF1DWUAZoWydCivZ,https://open.spotify.com/playlist/37i9dQZF1DWU...,Spotify,https://open.spotify.com/user/spotify,https://api.spotify.com/v1/users/spotify,spotify,user,spotify:user:spotify,https://api.spotify.com/v1/playlists/37i9dQZF1...,50
3,False,https://api.spotify.com/v1/playlists/1QxdL1fPW...,1QxdL1fPW4X4eex4CrFryW,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Good Mood,None,False,ODQwLDE2NDI4YzJhOGM1N2MxZjI1YWZmZGVjMTNhYWI0OT...,playlist,spotify:playlist:1QxdL1fPW4X4eex4CrFryW,https://open.spotify.com/playlist/1QxdL1fPW4X4...,Digster France,https://open.spotify.com/user/digster.fr,https://api.spotify.com/v1/users/digster.fr,digster.fr,user,spotify:user:digster.fr,https://api.spotify.com/v1/playlists/1QxdL1fPW...,101
4,False,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DX8CTD7HWYPDn,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Canta en la Ducha,None,False,MTU2NjIyOTIxMCwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMD...,playlist,spotify:playlist:37i9dQZF1DX8CTD7HWYPDn,https://open.spotify.com/playlist/37i9dQZF1DX8...,Spotify,https://open.spotify.com/user/spotify,https://api.spotify.com/v1/users/spotify,spotify,user,spotify:user:spotify,https://api.spotify.com/v1/playlists/37i9dQZF1...,50


In [11]:
users_playlists.groupby(['owner.id'])['tracks.total'].sum()

owner.id
1133054025                   50
cabina6                     150
csuberviola                 615
cuthbertgrau               1222
diestro                     136
digster.fr                  101
elcarlesdesiempre           270
elenanogues                 243
elmon                      1098
jahamer27                    73
joansayeras                 163
jrufi91                      92
martychristian             1831
mejoresplaylistsspotify     106
nerken                     1913
pcnaimad                     91
phobox                      452
pixelmaniatik               329
sonymusicentertainment       96
spotify                     305
tonijohan                  4390
travismartin86             2995
tv3alaxarxa                 221
wobniar                     121
ànima                        40
Name: tracks.total, dtype: int64

In [12]:
# remove all playlists that the owner is not the 'username'

owner_id = users_playlists[(users_playlists['owner.id'] != username)].index
 
users_playlists.drop(owner_id , inplace=True)



In [13]:
users_playlists['tracks.total'].sum()

1913

In [14]:
user_playlists = users_playlists.reset_index(drop=True)

In [15]:
user_playlists

,collaborative,href,id,images,name,primary_color,public,snapshot_id,type,uri,external_urls.spotify,owner.display_name,owner.external_urls.spotify,owner.href,owner.id,owner.type,owner.uri,tracks.href,tracks.total
0,False,https://api.spotify.com/v1/playlists/3ypuzkCtK...,3ypuzkCtKGOQ0Y63x295Jx,"[{'height': 640, 'url': 'https://mosaic.scdn.c...","Various Artists – Electro Swing Fever, Vol. 3",None,True,Miw1ZmMyMjZlYzlhYTVmZWViZDA1NWIzNzVmMzdjNzc0ZT...,playlist,spotify:playlist:3ypuzkCtKGOQ0Y63x295Jx,https://open.spotify.com/playlist/3ypuzkCtKGOQ...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/3ypuzkCtK...,60
1,False,https://api.spotify.com/v1/playlists/03aHsHAJv...,03aHsHAJv9vHySsb82CYB7,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",Bishop Allen,None,True,OCwxOTkxZmQ5MzM0NzUyMjc3NTFjYjI3NzI4MTVhNzIyZW...,playlist,spotify:playlist:03aHsHAJv9vHySsb82CYB7,https://open.spotify.com/playlist/03aHsHAJv9vH...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/03aHsHAJv...,85
2,False,https://api.spotify.com/v1/playlists/6uzNpJIT2...,6uzNpJIT2e1cOHVhYdg8Kr,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",ONE,None,True,MyxlYjI0MGViNWE2MjAzMGRmOWEwOThkYWY3NjNhYmJmNz...,playlist,spotify:playlist:6uzNpJIT2e1cOHVhYdg8Kr,https://open.spotify.com/playlist/6uzNpJIT2e1c...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/6uzNpJIT2...,1
3,False,https://api.spotify.com/v1/playlists/7uuoJ3glh...,7uuoJ3glhbTcKGo4PzqQu8,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Various Artists – Música Clássica: Mais Famosas,None,True,MyxkZGQ1YjI1MmJkZmIxNmYzZDBkMzU4MTRiZDg1YTM5MG...,playlist,spotify:playlist:7uuoJ3glhbTcKGo4PzqQu8,https://open.spotify.com/playlist/7uuoJ3glhbTc...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/7uuoJ3glh...,50
4,False,https://api.spotify.com/v1/playlists/3adsbUvUt...,3adsbUvUtQcVmqSdCpRNIH,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Nice COVERS,None,True,Myw2Y2FmN2FjNmYzODA1NGU4YjQzMTk2MWYwMTA1N2JlYW...,playlist,spotify:playlist:3adsbUvUtQcVmqSdCpRNIH,https://open.spotify.com/playlist/3adsbUvUtQcV...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/3adsbUvUt...,1
5,False,https://api.spotify.com/v1/playlists/5mLQ2pzeG...,5mLQ2pzeGmEwuc7NFrsmp6,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",FB,None,True,ODksY2UyNjhkYjgwYjk2MDc4MTNlZjBiMmJmNjg0Njg1NT...,playlist,spotify:playlist:5mLQ2pzeGmEwuc7NFrsmp6,https://open.spotify.com/playlist/5mLQ2pzeGmEw...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/5mLQ2pzeG...,81
6,True,https://api.spotify.com/v1/playlists/7nW9BlocQ...,7nW9BlocQCzpeLwufiJ4RJ,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",New Playlist,None,False,NCxmMWNiZjM2MmIzNzViZjViOTE1OTE4N2RhMjQxODVlMz...,playlist,spotify:playlist:7nW9BlocQCzpeLwufiJ4RJ,https://open.spotify.com/playlist/7nW9BlocQCzp...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,https://api.spotify.com/v1/playlists/7nW9BlocQ...,77
7,False,https://api.spotify.com/v1/playlists/28RqwFckD...,28RqwFckD9aUZqyksBUCKw,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",Reggeton,None,True,Niw1OWJmYWRkNjJhYzgxNjg4NTFhYTY2MDM5OTYwNTExZW...,playlist,spotify:playlist:28RqwFckD9aUZqyksBUCKw,https://open.spotify.com/playlist/28RqwFckD9aU...,Josep Foradada,https://open.spotify.com/user/nerken,https://api.s

In [16]:
user_playlists['id'][0]

'3ypuzkCtKGOQ0Y63x295Jx'

In [17]:
try:
    a = sp.user_playlist_tracks(user=username,playlist_id='03aHsHAJv9vHySsb82CYB7',limit=1,offset=0)
except:
    oauth2 = util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    token = util.prompt_for_user_token(username, scope=scope, client_id='22e782a724264812b124038bbf783396', client_secret='6219554f45764f2cbfe13306ec476f10', redirect_uri='https://localhost:8080')
    sp = spotipy.Spotify(auth=token)
    a = sp.user_playlist_tracks(user=username,playlist_id='03aHsHAJv9vHySsb82CYB7',limit=1,offset=0)

In [18]:
a.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [19]:
pd.Series(a)

href        https://api.spotify.com/v1/playlists/03aHsHAJv...
items       [{'added_at': '2017-03-29T18:14:26Z', 'added_b...
limit                                                       1
next        https://api.spotify.com/v1/playlists/03aHsHAJv...
offset                                                      0
previous                                                 None
total                                                      85
dtype: object

In [20]:
json_normalize(a)

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/03aHsHAJv...,"[{'added_at': '2017-03-29T18:14:26Z', 'added_b...",1,https://api.spotify.com/v1/playlists/03aHsHAJv...,0,None,85


In [21]:
a

{'href': 'https://api.spotify.com/v1/playlists/03aHsHAJv9vHySsb82CYB7/tracks?offset=0&limit=1',
 'items': [{'added_at': '2017-03-29T18:14:26Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/nerken'},
    'href': 'https://api.spotify.com/v1/users/nerken',
    'id': 'nerken',
    'type': 'user',
    'uri': 'spotify:user:nerken'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5rE9FxgMGmsZ8Cg4AeOavJ'},
       'href': 'https://api.spotify.com/v1/artists/5rE9FxgMGmsZ8Cg4AeOavJ',
       'id': '5rE9FxgMGmsZ8Cg4AeOavJ',
       'name': 'Bishop Allen',
       'type': 'artist',
       'uri': 'spotify:artist:5rE9FxgMGmsZ8Cg4AeOavJ'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',

In [22]:
tracks = pd.DataFrame(json_normalize(a['items'],max_level=2))

In [23]:
# display all columns without jupyter notebook limitation
()
pd.options.display.max_columns = None
display(tracks)

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2017-03-29T18:14:26Z,False,None,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/1D...,https://api.spotify.com/v1/albums/1D4pWDyvBQeq...,1D4pWDyvBQeq4x4UKTVvuI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Charm School,2003-01-01,day,13,album,spotify:album:1D4pWDyvBQeq4x4UKTVvuI,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,127666,False,False,ushm20322359,https://open.spotify.com/track/7c1QnSNfgHPLgyq...,https://api.spotify.com/v1/tracks/7c1QnSNfgHPL...,7c1QnSNfgHPLgyqUcdE1Il,False,Charm School,10,https://p.scdn.co/mp3-preview/8ae40ed9c2ee702d...,True,1,track,spotify:track:7c1QnSNfgHPLgyqUcdE1Il,None


In [24]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 40 columns):
added_at                              1 non-null object
is_local                              1 non-null bool
primary_color                         0 non-null object
added_by.external_urls.spotify        1 non-null object
added_by.href                         1 non-null object
added_by.id                           1 non-null object
added_by.type                         1 non-null object
added_by.uri                          1 non-null object
track.album.album_type                1 non-null object
track.album.artists                   1 non-null object
track.album.available_markets         1 non-null object
track.album.external_urls             1 non-null object
track.album.href                      1 non-null object
track.album.id                        1 non-null object
track.album.images                    1 non-null object
track.album.name                      1 non-null object
track

[{'added_at': '2017-03-29T18:14:26Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/nerken'},
   'href': 'https://api.spotify.com/v1/users/nerken',
   'id': 'nerken',
   'type': 'user',
   'uri': 'spotify:user:nerken'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5rE9FxgMGmsZ8Cg4AeOavJ'},
      'href': 'https://api.spotify.com/v1/artists/5rE9FxgMGmsZ8Cg4AeOavJ',
      'id': '5rE9FxgMGmsZ8Cg4AeOavJ',
      'name': 'Bishop Allen',
      'type': 'artist',
      'uri': 'spotify:artist:5rE9FxgMGmsZ8Cg4AeOavJ'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
   

In [26]:
tracks

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2017-03-29T18:14:26Z,False,None,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/1D...,https://api.spotify.com/v1/albums/1D4pWDyvBQeq...,1D4pWDyvBQeq4x4UKTVvuI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Charm School,2003-01-01,day,13,album,spotify:album:1D4pWDyvBQeq4x4UKTVvuI,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1,127666,False,False,ushm20322359,https://open.spotify.com/track/7c1QnSNfgHPLgyq...,https://api.spotify.com/v1/tracks/7c1QnSNfgHPL...,7c1QnSNfgHPLgyqUcdE1Il,False,Charm School,10,https://p.scdn.co/mp3-preview/8ae40ed9c2ee702d...,True,1,track,spotify:track:7c1QnSNfgHPLgyqUcdE1Il,None


In [27]:
json_normalize(tracks['track.album.artists'][0]).keys()

Index(['href', 'id', 'name', 'type', 'uri', 'external_urls.spotify'], dtype='object')

In [28]:
tracks2 = pd.DataFrame(json_normalize(tracks['track.album.artists'][0]))

In [29]:
tracks2

,href,id,name,type,uri,external_urls.spotify
0,https://api.spotify.com/v1/artists/5rE9FxgMGms...,5rE9FxgMGmsZ8Cg4AeOavJ,Bishop Allen,artist,spotify:artist:5rE9FxgMGmsZ8Cg4AeOavJ,https://open.spotify.com/artist/5rE9FxgMGmsZ8C...


In [130]:
def get_user_songs(username,user_playlists):
    '''
    Get all songs for all the playlists
    '''
    user_songs = pd.DataFrame()
    user_songs_raw = pd.DataFrame()
    total_num_of_tracks = 0
    user_songs['playlist_id'] = 1
    for x in range(len(user_playlists)):
        total_num_of_tracks += user_playlists['tracks.total'][x]
        playlist_id = user_playlists['id'][x]
        print(f'Playlist number {x+1}, id: {playlist_id}')
        limit = 100 # number of results per request 
        loops = math.ceil(user_playlists['tracks.total'][x]/limit)
        actual_num_of_rows = user_songs.shape[0]
        for x in range(loops):
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            try:
                results = sp.user_playlist_tracks(user=username,playlist_id=playlist_id,limit=100,offset=0+(x*100))
            except:
                oauth2 = util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
                token = util.prompt_for_user_token(username, scope=scope, client_id='22e782a724264812b124038bbf783396', client_secret='6219554f45764f2cbfe13306ec476f10', redirect_uri='https://localhost:8080')
                sp = spotipy.Spotify(auth=token)
                results = sp.user_playlist_tracks(user=username,playlist_id=playlist_id,limit=100,offset=0+(x*100))
            for x in range(len(results['items'])):
                a = (json_normalize(results['items'][x]))
                user_songs = user_songs.append(a, ignore_index=True)
        print(actual_num_of_rows)
        print(total_num_of_tracks)
        
        user_songs['playlist_id'].iloc[actual_num_of_rows:total_num_of_tracks] = playlist_id
    return user_songs #user_songs_raw
        

In [131]:
user_songs = get_user_songs(username,user_playlists)

Playlist number 1, id: 3ypuzkCtKGOQ0Y63x295Jx


C:\Users\Josep\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


0
60
Playlist number 2, id: 03aHsHAJv9vHySsb82CYB7


C:\Users\Josep\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


60
145
Playlist number 3, id: 6uzNpJIT2e1cOHVhYdg8Kr
145
146
Playlist number 4, id: 7uuoJ3glhbTcKGo4PzqQu8
146
196
Playlist number 5, id: 3adsbUvUtQcVmqSdCpRNIH
196
197
Playlist number 6, id: 5mLQ2pzeGmEwuc7NFrsmp6
197
278
Playlist number 7, id: 7nW9BlocQCzpeLwufiJ4RJ
278
355
Playlist number 8, id: 28RqwFckD9aUZqyksBUCKw
355
359
Playlist number 9, id: 17G5Gs4hjwytQt3l6mkph2
359
388
Playlist number 10, id: 4uBsB8VrlhHkN9Zazl0hO8
388
392
Playlist number 11, id: 7sKPU13AfyV0yN60OYFELl
392
442
Playlist number 12, id: 7bsAR19p7zO45OpIpiCqQB
442
464
Playlist number 13, id: 0K6zEWxGiCGZLejWC5RjLh
464
473
Playlist number 14, id: 3fFvSuiHKyiNHj4myybmRM
473
475
Playlist number 15, id: 6rcoq12RiupfqlzZK8rEX5
475
489
Playlist number 16, id: 64JY1fITMumLMHvbjyhYHS
489
504
Playlist number 17, id: 6F0bDRMilmN0VdkEtsS0Cz
504
523
Playlist number 18, id: 4YMWrJrGAyW04sfFwE3sc4
523
533
Playlist number 19, id: 7EHzH8D5Fv7EWf76tDTSvH
533
558
Playlist number 20, id: 0tiNVBKGtCjcH37FsQsaIg
558
563
Playlist n

In [132]:
user_songs

,added_at,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,is_local,playlist_id,primary_color,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.linked_from.external_urls.spotify,track.linked_from.href,track.linked_from.id,track.linked_from.type,track.linked_from.uri,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2018-10-24T14:52:55Z,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,False,3ypuzkCtKGOQ0Y63x295Jx,None,compilation,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/5VOdrHq3pi0FTHQ...,https://api.spotify.com/v1/albums/5VOdrHq3pi0F...,5VOdrHq3pi0FTHQZtBaj7V,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Electro Swing Fever, Vol. 3",2014-02-17,day,60.0,album,spotify:album:5VOdrHq3pi0FTHQZtBaj7V,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",1.0,181000.0,False,False,FR6F31402670,https://open.spotify.com/track/3wD62ofDBqgeJSW...,https://api.spotify.com/v1/tracks/3wD62ofDBqge...,3wD62ofDBqgeJSW2u7WKil,False,NaN,NaN,NaN,NaN,NaN,Ragtime Cat (feat. Lilja Bloom),21.0,https://p.scdn.co/mp3-preview/973b9b1ff20c6eba...,True,1.0,track,spotify:track:3wD62ofDBqgeJSW2u7WKil,None
1,2018-10-24T14:52:55Z,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,False,3ypuzkCtKGOQ0Y63x295Jx,None,compilation,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/5VOdrHq3pi0FTHQ...,https://api.spotify.com/v1/albums/5VOdrHq3pi0F...,5VOdrHq3pi0FTHQZtBaj7V,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Electro Swing Fever, Vol. 3",2014-02-17,day,60.0,album,spotify:album:5VOdrHq3pi0FTHQZtBaj7V,[{'external_urls': {'spotify': 'https://open.s...,[],1.0,265093.0,False,False,FR6F31402680,https://open.spotify.com/track/14EeHJP2ZhOqabZ...,https://api.spotify.com/v1/tracks/14EeHJP2ZhOq...,14EeHJP2ZhOqabZgCAF27r,False,NaN,NaN,NaN,NaN,NaN,Artichaut,0.0,None,True,2.0,track,spotify:track:14EeHJP2ZhOqabZgCAF27r,None
2,2018-10-24T14:52:55Z,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,False,3ypuzkCtKGOQ0Y63x295Jx,None,compilation,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/5VOdrHq3pi0FTHQ...,https://api.spotify.com/v1/albums/5VOdrHq3pi0F...,5VOdrHq3pi0FTHQZtBaj7V,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Electro Swing Fever, Vol. 3",2014-02-17,day,60.0,album,spotify:album:5VOdrHq3pi0FTHQZtBaj7V,[{'external_urls': {'spotify': 'https://open.s...,[],1.0,305280.0,False,False,FR6F31402690,https://open.spotify.com/track/4iGiK3PUx1r9G6N...,https://api.spotify.com/v1/tracks/4iGiK3PUx1r9...,4iGiK3PUx1r9G6N3vkQERe,False,NaN,NaN,NaN,NaN,NaN,Extra Mile,0.0,None,True,3.0,track,spotify:track:4iGiK3PUx1r9G6N3vkQERe,None
3,2018-10-24T14:52:55Z,https://open.spotify.com/user/nerken,https://api.spotify.com/v1/users/nerken,nerken,user,spotify:user:nerken,False,3ypuzkCtKGOQ0Y63x295Jx,None,compilation,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",https://open.spotify.com/album/5VOdrHq3pi0FTHQ...,https://api.spotify.com/v1/albums/5VOdrHq3pi0F...,5VOdrHq3pi0FTHQZtBaj7V,"[{'height': 640, 'url': 'ht

In [146]:
user_songs.to_csv(username+'_user_songs.csv')
user_playlists.to_csv(username+'_user_playlists.csv')

In [157]:
user_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 46 columns):
added_at                                   1913 non-null object
added_by.external_urls.spotify             1913 non-null object
added_by.href                              1913 non-null object
added_by.id                                1913 non-null object
added_by.type                              1913 non-null object
added_by.uri                               1913 non-null object
is_local                                   1913 non-null object
playlist_id                                1869 non-null object
primary_color                              0 non-null object
track.album.album_type                     1913 non-null object
track.album.artists                        1913 non-null object
track.album.available_markets              1913 non-null object
track.album.external_urls.spotify          1913 non-null object
track.album.href                           1913 non-null object


In [136]:
user_songs['track.name'] == 'tous les mêmes'

0       False
1       False
2       False
3       False
4       False
        ...  
1908    False
1909    False
1910    False
1911    False
1912    False
Name: track.name, Length: 1913, dtype: bool

In [139]:
user_songs.loc[user_songs['track.name'] == '3 Gymnopédies: No.1']

,added_at,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,is_local,playlist_id,primary_color,track.album.album_type,track.album.artists,track.album.available_markets,track.album.external_urls.spotify,track.album.href,track.album.id,track.album.images,track.album.name,track.album.release_date,track.album.release_date_precision,track.album.total_tracks,track.album.type,track.album.uri,track.artists,track.available_markets,track.disc_number,track.duration_ms,track.episode,track.explicit,track.external_ids.isrc,track.external_urls.spotify,track.href,track.id,track.is_local,track.linked_from.external_urls.spotify,track.linked_from.href,track.linked_from.id,track.linked_from.type,track.linked_from.uri,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
1512,1970-01-01T00:00:00Z,https://open.spotify.com/user/,https://api.spotify.com/v1/users/,,user,spotify:user:,False,2jig3jDOLAamg7rauyYbsW,None,compilation,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CH, C...",https://open.spotify.com/album/4EjKpnhFJf4s5Vv...,https://api.spotify.com/v1/albums/4EjKpnhFJf4s...,4EjKpnhFJf4s5Vvow7C1iz,"[{'height': 555, 'url': 'https://i.scdn.co/ima...",100 Classical Favourites,1998-01-01,day,100.0,album,spotify:album:4EjKpnhFJf4s5Vvow7C1iz,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CH, C...",3.0,292000.0,False,False,NLA509412615,https://open.spotify.com/track/2o9t1uqkAqXBOOf...,https://api.spotify.com/v1/tracks/2o9t1uqkAqXB...,2o9t1uqkAqXBOOfGrcpgGe,False,NaN,NaN,NaN,NaN,NaN,3 Gymnopédies: No.1,10.0,None,True,16.0,track,spotify:track:2o9t1uqkAqXBOOfGrcpgGe,None
